# MLP 실습

이번 실습에서는 PyTorch에서 MLP 및 backpropagation을 구현하여 XOR 문제에 학습해봅니다. 먼저 필요한 library들을 import합시다.

In [1]:
import torch
import numpy as np

그 다음 공유된 notebook과 똑같은 결과를 얻기 위한 코드를 구현합니다.

In [2]:
import random


seed = 7777

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

위의 코드에서는 seed 고정과 `cudnn.deterministic`, `cudnn.benchmark`를 조정하였습니다. 자세한 사항들은 https://pytorch.org/docs/stable/notes/randomness.html에서 찾아볼 수 있습니다.

다음은 이전 실습과 같은 코드로 XOR data를 생성하는 과정입니다.

In [3]:
x = torch.tensor([
    [0., 0.],
    [0., 1.],
    [1., 0.],
    [1., 1.]
])
y = torch.tensor([0, 1, 1, 0])

print(x.shape, y.shape)

torch.Size([4, 2]) torch.Size([4])


Data를 만들었으니 이제 MLP를 구현해야 합니다. 여기서 알아둬야할 점들은 다음과 같습니다:
- PyTorch에서는 우리가 학습하고자 하는 함수 $f$를 보통 `torch.nn.Module` class를 상속받은 class로 구현합니다.
- `torch.nn.Module` class는 abstract class로, `def forward`를 구현하도록 abstractmethod를 제공합니다. 이 method는 $f(x)$, 즉 함수의 출력에 해당됩니다.
- PyTorch에서는 선형함수를 `torch.nn.Linear` class로 구현할 수 있습니다.
- 마찬가지로 ReLU도 `torch.nn.ReLU`로 제공하고 있습니다.

위의 점들을 활용하여 2-layer MLP를 구현한 결과는 다음과 같습니다.

In [4]:
from torch import nn


class Model(nn.Module):
  def __init__(self, d, d_prime):
    super().__init__()

    self.layer1 = nn.Linear(d, d_prime)
    self.layer2 = nn.Linear(d_prime, 1)
    self.act = nn.ReLU()

  def forward(self, x):
    # x: (n, d)
    x = self.layer1(x)  # (n, d_prime)
    x = self.act(x)     # (n, d_prime)
    x = self.layer2(x)  # (n, 1)

    return x


model = Model(2, 10)

우리가 원하는 것은 $x \in \mathbb{R}^{n \times d}$가 입력으로 들어왔을 때 첫 번째 선형 함수를 거쳤을 때의 dimension이 $\mathbb{R}^{n \times d'}$이 되면서 두 번째 선형 함수를 거쳤을 때는 $\mathbb{R}^{n \times 1}$이 되기를 원합니다.
첫 번째 선형함수는 `self.layer1 = nn.Linear(d, d_prime)`로 구현할 수 있으며, `nn.Linear`의 첫 번째 인자는 입력으로 들어오는 tensor의 마지막 dimension, 두 번째 인자는 출력되는 tensor의 마지막 dimension을 뜻합니다.
이 정보를 이용하면 두 번째 선형함수도 `self.layer2 = nn.Linear(d_prime, 1)`로 구현할 수 있습니다.
마찬가지로 ReLU도 `nn.ReLU()`로 선언할 수 있습니다.

이제 $f(x)$에 대한 구현은 `def forward(self, x)`에서 할 수 있습니다.
생성자에서 선언한 세 개의 layer들을 순차적으로 지나면 우리가 원하던 결과를 얻을 수 있습니다.
여기서도 shape의 변화를 돌려보지 않고 예측하는 것이 실제로 구현하고 디버깅할 때 중요하게 여겨집니다.

마지막 줄에서는 입력 dimension이 2이고, 중간 dimension이 10이 되는 2-layer MLP에 대한 object를 생성하였습니다.
다음은 PyTorch에서 gradient descent를 어떻게 구현하는지 살펴봅시다.

In [5]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=0.1)

PyTorch는 다양한 update 알고리즘들을 `torch.optim`에서 제공하고 있습니다.
우리는 gradient descent를 사용할 것이기 때문에 `SGD` class를 불러옵니다.
`SGD`는 첫 번째 인자로 업데이트를 할 parameter들의 list를 받습니다. 예를 들어 선형 함수에서의 $w, b$가 있습니다.
PyTorch의 `nn.Module` class는 이러한 것들을 잘 정의해주기 때문에 `model.parameters()`의 형식으로 넘겨주기만 하면 됩니다.
두 번째 인자는 learning rate로, 이전의 gradient descent에서 사용하던 learning rate와 똑같은 역할을 가지고 있습니다.

다음은 실제 학습 코드를 구현하며, backpropagation이 어떻게 이루어지는지 살펴봅시다.

In [6]:
def train(n_epochs, model, optimizer, x, y):
  for e in range(n_epochs):
    model.zero_grad()

    y_pred = model(x)
    loss = (y_pred[:, 0] - y).pow(2).sum()

    loss.backward()
    optimizer.step()

    print(f"Epoch {e:3d} | Loss: {loss}")
  return model

`model(x)`를 통해 우리가 구현한 `forward` 함수를 사용할 수 있습니다. 그리고 이를 이용하여 MSE loss로 `model`을 평가할 수 있습니다.
Gradient descent의 구현은 다음 세 가지 부분에서 진행되고 있습니다:
- 본격적으로 학습이 진행되기 전에 `model.zero_grad()`를 실행합니다. 이는 각 parameter의 gradient 값이 저장되어 있을 수도 있기 때문에 지워주는 기능을 수행합니다.
- loss를 계산한 후, `loss.backward()`를 진행합니다. backward 함수를 실행하면 `model`에 있는 모든 parameter들의 `loss`에 대한 gradient를 계산하게 됩니다.
- 마지막으로 계산한 gradient들을 가지고 parameter들을 update하는 것은 `optimizer.step()`을 통해 진행하게 됩니다. optimizer는 이전에 인자로 받았던 `model.parameters()`에 해당하는 parameter들만 update를 진행하게 됩니다.

위의 코드로 학습을 진행한 코드는 다음과 같습니다.

In [7]:
n_epochs = 100
model = train(n_epochs, model, optimizer, x, y)

Epoch   0 | Loss: 2.8124639987945557
Epoch   1 | Loss: 2.309718132019043
Epoch   2 | Loss: 1.6600806713104248
Epoch   3 | Loss: 0.9176856279373169
Epoch   4 | Loss: 0.8310231566429138
Epoch   5 | Loss: 0.7857465147972107
Epoch   6 | Loss: 0.7469707131385803
Epoch   7 | Loss: 0.7105731964111328
Epoch   8 | Loss: 0.679701566696167
Epoch   9 | Loss: 0.6505993604660034
Epoch  10 | Loss: 0.6153720617294312
Epoch  11 | Loss: 0.5817742347717285
Epoch  12 | Loss: 0.5496706962585449
Epoch  13 | Loss: 0.5309392213821411
Epoch  14 | Loss: 0.500799834728241
Epoch  15 | Loss: 0.4737371802330017
Epoch  16 | Loss: 0.45748329162597656
Epoch  17 | Loss: 0.4339975118637085
Epoch  18 | Loss: 0.4096483290195465
Epoch  19 | Loss: 0.38796404004096985
Epoch  20 | Loss: 0.3663795590400696
Epoch  21 | Loss: 0.3485005497932434
Epoch  22 | Loss: 0.3266391158103943
Epoch  23 | Loss: 0.3139018714427948
Epoch  24 | Loss: 0.28671738505363464
Epoch  25 | Loss: 0.2610698640346527
Epoch  26 | Loss: 0.23623432219028473


Linear regression과는 달리 잘 수렴하는 모습을 보여주고 있습니다. 특히, 우리가 직접 gradient나 gradient descent를 구현하지 않아도 주어진 data를 잘 학습하는 코드를 PyTorch를 통해 구현할 수 있었습니다. 마지막으로 예측 결과를 살펴봅시다.

In [8]:
print(model(x))
print(y)

tensor([[0.0208],
        [1.0484],
        [1.0156],
        [0.0496]], grad_fn=<AddmmBackward0>)
tensor([0, 1, 1, 0])


매우 정확한 예측 결과를 보여주고 있습니다.